In [1]:
%env DEEPCELL_ACCESS_TOKEN=4UcGPTDd.VDOkiH1JZIWeHWkgHASN2IF1k1Y7fIZ5

env: DEEPCELL_ACCESS_TOKEN=4UcGPTDd.VDOkiH1JZIWeHWkgHASN2IF1k1Y7fIZ5


In [2]:
!pip install pytest
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install torchinfo

Looking in indexes: https://download.pytorch.org/whl/cpu


In [3]:
import pytest

import numpy as np

from unittest.mock import Mock

from tensorflow.python.platform import test

# from deepcell.model_zoo import PanopticNet
from temp_panoptic_orig import PanopticNet
from temp_mesmer import Mesmer
from deepcell.applications import MultiplexSegmentation
from deepcell.applications.mesmer import format_output_mesmer
from deepcell.applications.mesmer import mesmer_postprocess
from deepcell.applications.mesmer import mesmer_preprocess

In [4]:
class TestMesmer(test.TestCase):

    def test_mesmer_app(self):
        with self.cached_session():
            whole_cell_classes = [1, 3]
            nuclear_classes = [1, 3]
            num_semantic_classes = whole_cell_classes + nuclear_classes
            num_semantic_heads = len(num_semantic_classes)

            model = PanopticNet(
                'resnet50',
                input_shape=(256, 256, 2),
                norm_method=None,
                num_semantic_heads=num_semantic_heads,
                num_semantic_classes=num_semantic_classes,
                location=True,
                include_top=True,
                use_imagenet=False)

            model.summary()
            print(model.input_shape)

            input = np.random.rand(4, 256, 256, 2)
            output = model(input)
            print(output[0].shape)
            pred_out = model.predict(input, batch_size=4)

            temp_out = model(np.random.rand(4, 256, 256, 2))
            [print(i.shape) for i in temp_out]
            
            app = Mesmer(model)

            # test output shape
            shape = app.model.output_shape
            self.assertIsInstance(shape, list)
            self.assertEqual(len(shape), 4)

            # test predict with default
            x = np.random.rand(1, 500, 500, 2)
            y = app.predict(x)
            self.assertEqual(x.shape[:-1], y.shape[:-1])

            # test predict with nuclear compartment only
            x = np.random.rand(1, 500, 500, 2)
            y = app.predict(x, compartment='nuclear')
            self.assertEqual(x.shape[:-1], y.shape[:-1])
            self.assertEqual(y.shape[-1], 1)

            # test predict with cell compartment only
            x = np.random.rand(1, 500, 500, 2)
            y = app.predict(x, compartment='whole-cell')
            self.assertEqual(x.shape[:-1], y.shape[:-1])
            self.assertEqual(y.shape[-1], 1)

            # test predict with both cell and nuclear compartments
            x = np.random.rand(1, 500, 500, 2)
            y = app.predict(x, compartment='both')
            self.assertEqual(x.shape[:-1], y.shape[:-1])
            self.assertEqual(y.shape[-1], 2)

            # test that kwargs are passed through successfully
            app._predict_segmentation = Mock()

            # get defaults
            _ = app.predict(x, compartment='whole-cell')
            args = app._predict_segmentation.call_args[1]
            default_cell_kwargs = args['postprocess_kwargs']['whole_cell_kwargs']
            default_nuc_kwargs = args['postprocess_kwargs']['nuclear_kwargs']

            # change one of the args for each compartment
            maxima_threshold_cell = default_cell_kwargs['maxima_threshold'] + 0.1
            radius_nuc = default_nuc_kwargs['radius'] + 2

            _ = app.predict(x, compartment='whole-cell',
                            postprocess_kwargs_whole_cell={'maxima_threshold':
                                                           maxima_threshold_cell},
                            postprocess_kwargs_nuclear={'radius': radius_nuc})

            args = app._predict_segmentation.call_args[1]
            cell_kwargs = args['postprocess_kwargs']['whole_cell_kwargs']
            assert cell_kwargs['maxima_threshold'] == maxima_threshold_cell

            nuc_kwargs = args['postprocess_kwargs']['nuclear_kwargs']
            assert nuc_kwargs['radius'] == radius_nuc

            # check that rest are unchanged
            cell_kwargs['maxima_threshold'] = default_cell_kwargs['maxima_threshold']
            assert cell_kwargs == default_cell_kwargs

            nuc_kwargs['radius'] = default_nuc_kwargs['radius']
            assert nuc_kwargs == default_nuc_kwargs

            # test legacy version
            old_app = MultiplexSegmentation(model)

            # test predict with default
            x = np.random.rand(1, 500, 500, 2)
            y = old_app.predict(x)
            self.assertEqual(x.shape[:-1], y.shape[:-1])

            print("SUCCESS")

In [5]:
a = TestMesmer()
a.test_mesmer_app()

2024-09-12 07:14:36.152339: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-12 07:14:36.152587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-12 07:14:36.156651: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-12 07:14:36.156864: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-12 07:14:36.157073: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Model: "panopticnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_0 (InputLayer)           [(None, 256, 256, 2  0           []                               
                                )]                                                                
                                                                                                  
 location (Location2D)          (None, 256, 256, 2)  0           ['input_0[0][0]']                
                                                                                                  
 concatenate_location (Concaten  (None, 256, 256, 4)  0          ['input_0[0][0]',                
 ate)                                                             'location[0][0]']               
                                                                                        

2024-09-12 07:14:37.846864: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


(4, 256, 256, 1)
(4, 256, 256, 1)
(4, 256, 256, 3)
(4, 256, 256, 1)
(4, 256, 256, 3)
SUCCESS
